# Trabajo en clase 01 - Javier Fernández (2020425930)
## Series de Fourier

- Realice las actividades que se le indican en los espacios disponibles. 
- Debe entregar una copia de este cuaderno en el espacio habilitado el tecDigital para el trabajo en clase 02.
- **Valor total 12 puntos**.
- Puede agregar celdas de texto o código si lo considera necesario.
- Use nombres descriptivos para las variables y las funciones que defina. 
- Agregue todos los comentarios que considere necesarios para mejorar la comprensión del código.

## Series de Fourier

Para una función que cumple que $f(t+T) = f(t)$, es decir, una función periódica, se puede definir una frecuencia principal $\omega = \omega_1 = \dfrac{2\pi}{T}$

Esta función se puede expresar como una expansión de la forma:
\begin{equation}
f(t) = \dfrac{a_0}{2} + \sum_{n=1}^{\infty}a_n\cos{(n\omega t)} + \sum_{n=1}^{\infty}b_n\sin{(n\omega t)}
\end{equation}

donde la intensidad de cada frecuencia se calcula como $a_n^2 + b_n^2$

Cada coeficiente se puede calcular de la siguiente manera:

\begin{align}
a_n &= \dfrac{2}{T}\int_0^T\cos{(n\omega t)}f(t)\mathrm{d}t\\
b_n &= \dfrac{2}{T}\int_0^T\sin{(n\omega t)}f(t)\mathrm{d}t
\end{align}

donde $a_0 = 2\langle f(t)\rangle$

Note que en caso de que la función que se quiere aproximar esté representada por una señal (valores para una serie de instantes de tiempo), la suma de Fourier tiene que hacerse para cada instante _t_. Esto implica que los coeficientes $a_n$ y $b_n$ también deben calcularse para cada _t_.


### Actividad 
Genere un gráfico interactivo que aproxime la señal con una serie de Fourier para una cantidad de términos arbitraria.

Se van a crear dos funciones: una calcula los coeficientes $a_n$ y $b_n$ y la otra calcula la suma de Fourier para cada _t_

Primero genere la función que retorna la suma de Fourier: **Valor: 4 puntos**
1. Inicialice un arreglo lleno de ceros llamado suma_Fourier. En este arreglo se almacenarán los valores de la serie de Fourier para cada instante _t_.
2. Inicialice un ciclo que va a iterar sobre la cantidad de términos de la suma.
    
    2.1. Calcule los coeficientes a_n y b_n para el n-ésimo término de la suma. Esta parte se realiza con la función que se describe en la próxima parte.
    
    2.2. Calcule el valor de la suma de Fourier para cada _t_.   
    2.3. Sume el valor obtenido en 2.2 al arreglo suma_Fourier

3. Sume a_0/2 a cada entrada de suma_Fourier
4. Retorne el arreglo suma_Fourier

In [1]:
# Bibliotecas básicas
import numpy as np
from scipy import signal
from scipy import integrate
import matplotlib.pyplot as plt


# funciones para la visualización interactiva
from ipywidgets import interactive, interact, FloatSlider
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Función suma de Fourier
def aproxFourier(arreglo_X,arreglo_Y,nTerminos, nMuestras, periodoL):
    
    # Creación del array de ceros para almacenar los valores de las iteraciones
    suma_Fourier = np.zeros([1,len(arreglo_X)])[0]
    omega= 2*np.pi / periodoL


    # Primer término a0
    intervaloInt= np.linspace(0,periodoL,nMuestras)
    a0 = 2 * (1/periodoL) * integrate.simpson(arreglo_Y,intervaloInt)
    suma_Fourier += a0/2

    # Resto de términos hasta el término arbitrario definido
    for n in range(1,nTerminos+1):
        a,b = calcAB(arreglo_X, arreglo_Y, n, nMuestras, periodoL, omega, intervaloInt)
        f_aprox = a*np.cos(n*omega*arreglo_X) + b*np.sin(n*omega*arreglo_X)
        suma_Fourier += f_aprox
    
    return suma_Fourier

Ahora escriba la función que encuentre los valores de $a_n$ y $b_n$ para usar en la función anterior: **Valor: 4 puntos**
1. Esta función calcula los coeficientes $a_n$ y $b_n$ para un único _n_. Es decir, solo retorna 2 valores. 
2. Calcule los valores de los coeficientes $a_n$ y $b_n$ para cada _t_. Utilice la función `scipy.integrate.simpson()` para calcular las integrales correspondientes.
3. Retorne los coeficientes $a_n$ y $b_n$.

In [3]:
# Función cálculo de coeficientes a_n y b_n

def calcAB(arreglo_X, arreglo_Y, nT, nMuestras, periodoL, omega, intervaloInt):
    # Se define la función a integrar para obtener el coeficiente a
    a = (2/periodoL) * integrate.simpson(np.cos(nT*omega*arreglo_X)*arreglo_Y,intervaloInt)


    # Se define la función a integrar para obtener el coeficiente b
    b = (2/periodoL) * integrate.simpson( np.sin(nT*omega*arreglo_X)*arreglo_Y,intervaloInt)


    return a,b

Finalmente, genere un gráfico interactivo que use las funciones anteriores para aproximar una señal de entrada para una cantidad de términos en la suma de Fourier arbitraria. **Valor: 4 puntos**

In [4]:
def graficoFourier(nTerminos):
    # Parámetros tiempo
    periodoL = 8 # periodo de la función f(x)
    nMuestras = 1000
    nTerminos = int(nTerminos)
    
    # Parámetros de las señales de entrada
    freq = 4 # frecuencia angular o número de ondas en un periodo
    dutycycle = 0.5
    width_range = 1


    # Puntos t
    arreglo_X = np.linspace(0, periodoL, nMuestras, endpoint=False)
        
    # Señales y(t)
    # arreglo_Y = np.sin(2*np.pi*1*arreglo_X)
    # arreglo_Y = np.cos(2*np.pi*1*arreglo_X)
    # arreglo_Y = signal.sawtooth(2.0*np.pi*arreglo_X*freq/periodoL ,width=width_range)
    # arreglo_Y = signal.square(2*np.pi*arreglo_X*freq/periodoL , duty=dutycycle)
    # arreglo_Y = signal.gausspulse(2.0*np.pi*arreglo_X*freq/periodoL, fc=0.05)
    arreglo_Y = signal.ricker(nMuestras, 100) # Sombrero mexicano
    # arreglo_Y = signal.unit_impulse(nMuestras, 'mid')

    # Aproximación por Fourier
    señal_aprox = aproxFourier(arreglo_X,arreglo_Y,nTerminos, nMuestras, periodoL)



    fig, ax = plt.subplots(dpi=120)
    
    ax.plot(arreglo_X, señal_aprox, label='Aprox Fourier')
    ax.plot(arreglo_X, arreglo_Y, label='Señal orig', alpha=0.5)
  
    ax.set_xlabel('$t$')
    ax.set_ylabel('$y=f(t)$')
    # ax.set_xlim([2,6])

    
    ax.legend(loc='best', prop={'size':10})
    ax.set_title('Análisis de Fourier para una señal')
    
    plt.show()

In [5]:
# Gráfico 

nT = FloatSlider(min=1, max=100, step=1., continuous_update=False)
interact(graficoFourier, nTerminos = nT)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='nTerminos', min=1.0, step=1…

<function __main__.graficoFourier(nTerminos)>